In [5]:
import pandas as pd
from collections import defaultdict
import re

In [ ]:
## allgemeine Funktionen

def make_nested_frame(dict) -> pd.DataFrame:
    """
    Die Funktion nimmt ein dreifach verschachteltes Dict und generiert daraus ein DataFrame mit dreifachem index
    """
    return pd.DataFrame.from_dict({(k1, k2, k3): v3 
      for k1, v1 in dict.items() 
      for k2, v2 in v1.items()
      for k3, v3 in v2.items()},
      orient='index', dtype='object')

def feldauswertung(kategorie: str, inhalt: str) -> list[tuple[str,str]]:
    """
    Die Funktion hat zwei Argumente: die Pica-Kategorie und den Feldinhalt.
    Per regex wird das gewünschte Unterfeld gesucht und in einem tuple mit der Klarfeldbezeichnung ausgegeben.
    Weil auch Unterfelder wiederholbar sind, kommt eine Liste von Tupeln zurück.
    """
    ergebnisse = list()
    matchlist = [
        ('209A','f','standort'),
        ('209A','g','signatur_g'),
        ('209A','d','ausleihcode'),
        ('209A','a','signatur_a'),
        ('209A','c','sig_komm'),
        ('209C','a','akz'),
        ('237A','a','f4801_a'),
        ('237A','k','f4801_k'),
        ('247C','9','bibliothek')]

    for match in matchlist:
        if kategorie == match[0]:        
            ergebnisse.append((match[2],'; '.join(re.findall(f"\${match[1]}([^\$]+)", inhalt))))
    
    return ergebnisse


def get_exemplare(dateiname: str) -> pd.DataFrame:
    """
    Auf IDN und Exemplardatenfelder reduzierter Dump, dessen Dateiname als Argument übergeben wird, wird in df mit gewünschten Feldern umgewandelt
    """
    with open(f'../raw/{dateiname}', 'r') as f:
        df = pd.DataFrame()
        for l in f:
            # print(l)
            line = l.split(' ',1)
            if l == '\n':
                df = pd.concat([df,make_nested_frame(exemplare)])

            elif line[0] == '003@':
                exemplare = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda:defaultdict(str))))
                idn = line[1][2:].strip()
            elif line[0] == '101@':
                einrichtung = line[1][2:].strip()
            
            else:
                feld = line[0].split('/')
                feldergebnis = feldauswertung(feld[0],line[1].strip())
                for ergebnis in feldergebnis:
                    # der seltsame join deshalb, weil Felder auch wiederholt werden können und die Ergebnisse dann zusammengezogen und durch Semikolon getrennt werden. wahrscheinlich nicht die beste lösung.
                    exemplare[idn][einrichtung][feld[1]][ergebnis[0]] = '; '.join([ergebnis[1],exemplare[idn][einrichtung][feld[1]][ergebnis[0]]])  
        
        df.index.set_names(['idn','einrichtung','exemplar'], inplace=True)
        return df
 
def get_exemplare_neu(dateiname: str) -> pd.DataFrame:
    with open(f'../raw/{dateiname}', 'r') as f:
        for l in f:
            line = l.split(' ',1)
            if l == '\n':
                # idn-satz schreiben
            elif line[0] == '003@':
                # idn setzen
                idn = line[1][2:].strip()
                
            elif line[0] == '101@':
                # einrichtung setzen
                einrichtung = line[1][2:].strip()
            else:
                # feld setzen
                feld = line[0].split('/')
                feldergebnis = feldauswertung(feld[0],line[1].strip())
                for ergebnis in feldergebnis:
                    

            



In [2]:
df = pd.DataFrame(index=['idn','einrichtung','exemplar'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, idn to exemplar
Empty DataFrame

In [3]:
df

""
idn
einrichtung
exemplar
